# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fcbd0e1da60>
├── 'a' --> tensor([[-0.6533, -0.7814,  0.8347],
│                   [-0.1776, -0.8544, -1.2637]])
└── 'x' --> <FastTreeValue 0x7fcbd0e1db20>
    └── 'c' --> tensor([[-0.0710, -0.4278, -0.0236,  0.1338],
                        [-1.2858, -1.6180, -0.1188,  1.0551],
                        [-0.7933, -0.5985, -0.6393, -1.7236]])

In [4]:
t.a

tensor([[-0.6533, -0.7814,  0.8347],
        [-0.1776, -0.8544, -1.2637]])

In [5]:
%timeit t.a

86 ns ± 0.112 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fcbd0e1da60>
├── 'a' --> tensor([[ 0.5286, -0.5614,  0.5087],
│                   [-0.2600,  0.1510,  0.0213]])
└── 'x' --> <FastTreeValue 0x7fcbd0e1db20>
    └── 'c' --> tensor([[-0.0710, -0.4278, -0.0236,  0.1338],
                        [-1.2858, -1.6180, -0.1188,  1.0551],
                        [-0.7933, -0.5985, -0.6393, -1.7236]])

In [7]:
%timeit t.a = new_value

93.2 ns ± 0.113 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.6533, -0.7814,  0.8347],
               [-0.1776, -0.8544, -1.2637]]),
    x: Batch(
           c: tensor([[-0.0710, -0.4278, -0.0236,  0.1338],
                      [-1.2858, -1.6180, -0.1188,  1.0551],
                      [-0.7933, -0.5985, -0.6393, -1.7236]]),
       ),
)

In [10]:
b.a

tensor([[-0.6533, -0.7814,  0.8347],
        [-0.1776, -0.8544, -1.2637]])

In [11]:
%timeit b.a

80.8 ns ± 0.182 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.9798, -0.9849,  0.7684],
               [ 0.0503,  0.6477,  1.1504]]),
    x: Batch(
           c: tensor([[-0.0710, -0.4278, -0.0236,  0.1338],
                      [-1.2858, -1.6180, -0.1188,  1.0551],
                      [-0.7933, -0.5985, -0.6393, -1.7236]]),
       ),
)

In [13]:
%timeit b.a = new_value

657 ns ± 2.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.11 µs ± 3.59 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.7 µs ± 23.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

172 µs ± 1.32 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

173 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fcafa2affd0>
├── 'a' --> tensor([[[-0.6533, -0.7814,  0.8347],
│                    [-0.1776, -0.8544, -1.2637]],
│           
│                   [[-0.6533, -0.7814,  0.8347],
│                    [-0.1776, -0.8544, -1.2637]],
│           
│                   [[-0.6533, -0.7814,  0.8347],
│                    [-0.1776, -0.8544, -1.2637]],
│           
│                   [[-0.6533, -0.7814,  0.8347],
│                    [-0.1776, -0.8544, -1.2637]],
│           
│                   [[-0.6533, -0.7814,  0.8347],
│                    [-0.1776, -0.8544, -1.2637]],
│           
│                   [[-0.6533, -0.7814,  0.8347],
│                    [-0.1776, -0.8544, -1.2637]],
│           
│                   [[-0.6533, -0.7814,  0.8347],
│                    [-0.1776, -0.8544, -1.2637]],
│           
│                   [[-0.6533, -0.7814,  0.8347],
│                    [-0.1776, -0.8544, -1.2637]]])
└── 'x' --> <FastTreeValue 0x7fcbd0e2f790>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

40 µs ± 139 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fcaf1eeb1c0>
├── 'a' --> tensor([[-0.6533, -0.7814,  0.8347],
│                   [-0.1776, -0.8544, -1.2637],
│                   [-0.6533, -0.7814,  0.8347],
│                   [-0.1776, -0.8544, -1.2637],
│                   [-0.6533, -0.7814,  0.8347],
│                   [-0.1776, -0.8544, -1.2637],
│                   [-0.6533, -0.7814,  0.8347],
│                   [-0.1776, -0.8544, -1.2637],
│                   [-0.6533, -0.7814,  0.8347],
│                   [-0.1776, -0.8544, -1.2637],
│                   [-0.6533, -0.7814,  0.8347],
│                   [-0.1776, -0.8544, -1.2637],
│                   [-0.6533, -0.7814,  0.8347],
│                   [-0.1776, -0.8544, -1.2637],
│                   [-0.6533, -0.7814,  0.8347],
│                   [-0.1776, -0.8544, -1.2637]])
└── 'x' --> <FastTreeValue 0x7fcaf1eeb100>
    └── 'c' --> tensor([[-0.0710, -0.4278, -0.0236,  0.1338],
                        [-1.2858, -1.6180, -0.1188,  1.0551],
                 

In [23]:
%timeit t_cat(trees)

37.3 µs ± 172 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

75.2 µs ± 96.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.6533, -0.7814,  0.8347],
                [-0.1776, -0.8544, -1.2637]],
       
               [[-0.6533, -0.7814,  0.8347],
                [-0.1776, -0.8544, -1.2637]],
       
               [[-0.6533, -0.7814,  0.8347],
                [-0.1776, -0.8544, -1.2637]],
       
               [[-0.6533, -0.7814,  0.8347],
                [-0.1776, -0.8544, -1.2637]],
       
               [[-0.6533, -0.7814,  0.8347],
                [-0.1776, -0.8544, -1.2637]],
       
               [[-0.6533, -0.7814,  0.8347],
                [-0.1776, -0.8544, -1.2637]],
       
               [[-0.6533, -0.7814,  0.8347],
                [-0.1776, -0.8544, -1.2637]],
       
               [[-0.6533, -0.7814,  0.8347],
                [-0.1776, -0.8544, -1.2637]]]),
    x: Batch(
           c: tensor([[[-0.0710, -0.4278, -0.0236,  0.1338],
                       [-1.2858, -1.6180, -0.1188,  1.0551],
                       [-0.7933, -0.5985, -0.6393, -1.7236]],
         

In [26]:
%timeit Batch.stack(batches)

99.2 µs ± 232 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.6533, -0.7814,  0.8347],
               [-0.1776, -0.8544, -1.2637],
               [-0.6533, -0.7814,  0.8347],
               [-0.1776, -0.8544, -1.2637],
               [-0.6533, -0.7814,  0.8347],
               [-0.1776, -0.8544, -1.2637],
               [-0.6533, -0.7814,  0.8347],
               [-0.1776, -0.8544, -1.2637],
               [-0.6533, -0.7814,  0.8347],
               [-0.1776, -0.8544, -1.2637],
               [-0.6533, -0.7814,  0.8347],
               [-0.1776, -0.8544, -1.2637],
               [-0.6533, -0.7814,  0.8347],
               [-0.1776, -0.8544, -1.2637],
               [-0.6533, -0.7814,  0.8347],
               [-0.1776, -0.8544, -1.2637]]),
    x: Batch(
           c: tensor([[-0.0710, -0.4278, -0.0236,  0.1338],
                      [-1.2858, -1.6180, -0.1188,  1.0551],
                      [-0.7933, -0.5985, -0.6393, -1.7236],
                      [-0.0710, -0.4278, -0.0236,  0.1338],
                      [-1.2858, -

In [28]:
%timeit Batch.cat(batches)

180 µs ± 287 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

446 µs ± 6.04 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
